In [5]:
import os
import torch
import transformers
import peft
import datasets
import evaluate
import time
import re
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
dataset_path = "/app/datasets/oberon/docs/bb_ru"
fixed_dataset_path = "/app/datasets/oberon/docs/bb_ru_formatted"

In [3]:
model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"

In [6]:
def load_texts():
    file_names = [] 
    for subdir, dirs, files in os.walk(dataset_path):
        if "/Rsrc/" in subdir:
            continue
        for file in files:
            file_names.append(os.path.join(subdir, file))
    texts = []
    for f in file_names:
        with open(f, 'r', encoding='utf-8') as file:
            text = file.read()
            text = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', text)
            texts.append(text)
    return file_names,texts
names, texts = load_texts()

In [7]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [8]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForCausalLM.from_pretrained(model_name, device_map=device,quantization_config=bnb_config,)

Loading checkpoint shards: 100%|██████████| 4/4 [02:01<00:00, 30.37s/it]


In [10]:
system="Перепеши документ в формате markdown. Будь точен, ничего не изменяй. Выделяй найденные блоки кода. Отформатируй код табуляцией"

In [11]:
def rewrite(text):
    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": text}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)
    
    # Directly use generate() and tokenizer.decode() to get the output.
    # Use `max_new_tokens` to control the maximum output length.
    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=4096,
        temperature = 0.1
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response

In [ ]:
for i in range(len(names)):
    out_name = names[i].replace(dataset_path, fixed_dataset_path) + '.md'
    if os.path.isfile(out_name):
        print('already done ' + out_name)
        continue
    os.makedirs(os.path.dirname(out_name), exist_ok=True)
    text = rewrite(texts[i])
    print(out_name)
    with open(out_name, 'w', encoding='utf-8') as f:
        f.write(text)

already done /app/datasets/oberon/docs/bb_ru_formatted/Empty.odc.md
already done /app/datasets/oberon/docs/bb_ru_formatted/i21Greeting.odc.md
already done /app/datasets/oberon/docs/bb_ru_formatted/i21Привет.odc.md
/app/datasets/oberon/docs/bb_ru_formatted/readme, профи!.odc.md


In [32]:
from IPython.display import display, Markdown, Latex
display(Markdown(response))

```markdown
# Соглашения по оформлению программ
## ENGLISH
Редакция перевода: Ф.В.Ткачев, 2009-03-20

## Содержание

1. Самое важное
2. Оберон
3. Элементарные типы
4. Шрифтовые атрибуты
5. Комментарии
6. Точки с запятой
7. Разыменование
8. Регистр
9. Имена
10. Пробелы
11. Пример
12. Заголовок Открытого исходного кода

Этот текст описывает соглашения относительно правил программирования и форматирования исходного кода, которые использовались при создании системы Блэкбокс (BlackBox Component Framework). По документации существуют аналогичные соглашения.

Некоторые правила программирования важнее остальных. В первом разделе описаны самые важные правила. Остальные разделы содержат правила косметического характера, описывающие, как выглядят программы, опубликованные компанией Oberon microsystems. Если эти правила вам нравятся, смело пользуйтесь ими. Они могут сделать ваши программы более легкими для понимания теми, кто будет использовать их для проектирования, документирования и кодирования в BlackBox.

## 1. Самое важное

Наиболее важные соглашения по программированию нацелены на то, чтобы обеспечить развиваемость (evolvability) программ. Это означает, что нужно максимально облегчить внесение надежных изменений в существующие программы, даже если программы были написаны давно или кем-то другим. Развиваемость часто может быть повышена путем увеличения степени локализованности частей программы: если можно четко ограничить тот отрезок программного текста, на который могут повлиять изменения в исправляемой части программы, то легче определить всю цепочку необходимых дальнейших изменений. В сущности, речь здесь о том, чтобы удержать под контролем всю «цепную реакцию» изменений.

Предусловия (preconditions) — один из наиболее полезных инструментов для обнаружения непредусмотренных последствий «цепной реакции» изменений. Проверки предусловий позволяют «отловить» семантические ошибки на максимально ранней стадии, то есть максимально близко к их источникам. Хорошо расставленные проверки (операторы ASSERT) могут радикально сократить время отладки после больших изменений в проекте.

Везде, где можно, используйте статические средства, чтобы выразить все, что вам известно о структуре программы. В частности, используйте для этой цели систему типов и модули Компонентного Паскаля, чтобы компилятор смог помочь вам выявить несоответствия и, тем самым, стать эффективным инструментом реструктуризации.

Выполняйте проверки предусловий. Не позволяйте вызывающей программе «входить» в ваш модуль, если не выполнены предусловия входных точек вашего модуля (процедур, методов). Таким образом вы избежите распространения чужих ошибок в вашем собственном коде.

Используйте следующие номера в соответствии с принятыми в Блэкбоксе правилами:
- Свободные: 0 .. 19 — используется для временных точек останова
- Предусловия: 20 .. 59 — допустимость параметров на входе процедуры
- Постусловия: 60 .. 99 — допустимость результатов в конце процедуры
- Инварианты: 100 .. 120 — допустимость промежуточных состояний (выявление локальных ошибок)
- Резерв: 121 .. 125 — зарезервировано на будущее
- Еще не реализовано: 126 — процедура еще не реализована
- Резерв: 127 — зарезервировано на будущее

Должно быть как можно меньше глобальных переменных.

Значения глобальных переменных могут изменяться из любого места программы, в любое время. Это затрудняет отслеживание всех возможных взаимодействий (т.н. «побочных эффектов») с такими переменными. Тем самым увеличивается вероятность внесения ошибок при изменении их использования.

Процедуры-функции, т.е. процедуры, возвращающие результат, не должны менять глобальные переменные или VAR параметры в качестве побочного эффекта. Легче иметь дело с такими функциями, которые являются истинными функциями в математическом смысле, т.е. не имеют побочных эффектов. Но допустимо вместе с возвращением результата функции присваивать значения OUT параметрам.

Используйте ELSE в CASE или WITH, только если входные данные допускают расширение. Ветку ELSE в операторе CASE x OF или WITH x DO следует использовать, только если x действительно является расширяемым. Если x не расширяемо, то все возможные варианты должны быть известны статически и могут быть перечислены в списке альтернатив.

## 2. Оберон

Не используйте в новых программах специальные функции языка Оберон, которые не являются частью Компонентного Паскаля. В частности, это тип LONGREAL и процедура COPY. Старайтесь избегать супервызовов (используйте композицию вместо наследования) и процедурных типов (используйте вместо них объекты и методы).

## 3. Элементарные типы

По умолчанию используйте типы INTEGER, REAL и CHAR. Типы BYTE, SHORTINT, LONGINT, SHORTREAL и SHORTCHAR следует использовать только тогда, когда для этого есть веские причины. Эти вспомогательные типы введены для обеспечения взаимодействия с другими программами, для упаковки очень больших структур или для вычисления очень больших целых чисел. Это правило, конечно, важнее для экспортируемых интерфейсов, чем для скрытых реализаций.

## 4. Шрифтовые атрибуты

Программы пишутся как «плоский» текст, цветом, заданным «по умолчанию», со следующими исключениями:
- Комментарии пишутся курсивом: `(* это комментарий *)`
- Экспортированные элементы, за исключением полей записи и сигнатур методов в объявлении записей, пишутся жирным шрифтом: `PROCEDURE Do*;`
- Ключевые слова, соответствующие нелокальной передаче управления, пишутся жирным шрифтом, например, RETURN, EXIT и HALT
- Текстовые фрагменты, которые тестируются и подвержены изменениям, временно могут быть записаны другим цветом

## 5. Комментарии

Комментарии, которые являются частью описания интерфейса (в отличие от простого описания реализации), оформляются дополнительными звездочками, своего рода «метками экспорта» для комментариев, например, `(** охрана для команды xyz **)`

## 6. Точки с запятой

Точки с запятой используются для разделения операторов, а не для их завершения. Это означает, что не должно быть избыточных точек с запятой.

Хорошо:
```
IF done THEN
    Print(result)
END
```

Плохо:
```
IF done THEN
    Print(result);
END
```

## 7. Разыменование

Оператор разыменования ^ следует убирать отовсюду, где это возможно.

Хорошо:
```
h.next := p.prev.next
```

Плохо:
```
h^.next := p^.prev^.next
```

## 8. Регистр

Вообще говоря, каждый идентификатор начинается с маленькой буквы, за исключением следующих случаев:
- Имя модуля всегда начинается с большой буквы
- Имя типа всегда начинается с большой буквы
- Имя процедуры всегда начинается с большой буквы, это верно также для процедурных констант, типов, переменных, параметров и полей записи.

Хорошо:
```
null = 0X;
DrawDot = PROCEDURE (x, y: INTEGER);
PROCEDURE Proc (i, j: INTEGER; Draw: DrawDot);
```

Плохо:
```
NULL = 0X;
PROCEDURE isEmpty (q: Queue): BOOLEAN;
R = RECORD
    draw: DrawDot
END;
```

Не используйте идентификаторы (кроме однобуквенных), состоящих только из заглавных букв. Их следует зарезервировать для языка.

## 9. Имена

Для имен обычных процедур берутся глаголы, например, РисоватьТочку (DrawDot)

Для имен процедур-функций используются существительные или предикаты, например, NewObject(), IsEmpty(q)

Тип запись объявляйте только при необходимости (FooDesc for each Foo are normally not needed anymore)

Процедуры с именами на Init обладают тем свойством, что при повторном вызове они либо ничего не делают, либо приводят к аварийной остановке. Напротив, процедуры для многократной (пере)установки состояния начинаются префиксом Set.

Примеры:
```
PROCEDURE InitDir (dir: Directory);
PROCEDURE (p: Port) Init (unit, colors: LONGINT);
```

Процедуры-охранники имеют те же имена, что и охраняемые команды, но с суффиксом Guard.

Пример:
```
PROCEDURE PasteChar;
PROCEDURE PasteCharGuard (VAR par: Dialog.Par);
```

## 10. Пробелы

Новый уровень отступа создается добавлением еще одного символа табуляции

Между элементами списков `<объявлений>`, между фактическими параметрами и между операторами вставляется один пробел.

Хорошо:
```
VAR a, b, c: INTEGER;
DrawRect(l, t, r, b);
a := i * 8 + j - m[i, j];
```

Плохо:
```
VAR a,b,c: INTEGER;
DrawRect(l,t,r,b);
a:=b;
a := i*8 + j - m[i,j];
```

Ставится один пробел между именем процедуры (или ее меткой экспорта) и списком параметров в описании, но не в вызове

Хорошо:
```
PROCEDURE DrawDot* (x, y: INTEGER);
DrawDot(3, 5);
```

Плохо:
```
PROCEDURE DrawDot*(x, y: INTEGER);
DrawDot (3, 5);
```

Открывающие и закрывающие ключевые слова стоят на одном уровне отступа либо записаны в одной строке

Секции IMPORT, CONST, TYPE, VAR, PROCEDURE имеют стоят на один отступ глубже по сравнению с уровнем окружения.

Цикл LOOP никогда не пишется в одной строке

PROCEDURE X и END X всегда выровнены

Если вся конструкция не помещается в одной строке, за ключевым словом не ставится описание типа или оператор

Содержимое операторов IF, WHILE, REPEAT, LOOP, FOR, CASE стоит на один отступ глубже, если оно не помещается в одной строке.

Хорошо:
```
IF expr THEN S0 ELSE S1 END;
REPEAT S0 UNTIL expr;
WHILE expr DO S0 END;
IF expr THEN
    S0
ELSE
    S1
END;
REPEAT
    S0
UNTIL expr;
LOOP
    S0;
    IF expr THEN EXIT END;
    S1
END;
i := 0; WHILE i # 15 DO DrawDot(a, i); INC(i) END;
TYPE View = POINTER TO RECORD (Views.ViewDesc) END;
IMPORT Views, Containers,
TextModels, TextViews;
VAR a, b: INTEGER;
s: TextMappers.Scanner;
```

Плохо:
```
IF expr THEN S0
ELSE S1 END;
PROCEDURE P;
BEGIN ... END P;
BEGIN i := 0;
j := a + 2;
...
REPEAT i := 0;
j := a + 2;
...
```

## 11. Пример

MODULE StdExample;

IMPORT Models, Views, Controllers;

CONST null = 0X;

TYPE
View* = POINTER TO RECORD (Views.View)
a*, b*: INTEGER
END;

VAR view-: View;

PROCEDURE Calculate* (x, y: INTEGER);
VAR area: LONGINT;

PROCEDURE IsOdd (x: INTEGER): BOOLEAN;
BEGIN
RETURN ODD(x)
END IsOdd;

BEGIN
area := x * y;
IF IsOdd(area) THEN area := 1000 END
END Calculate;

BEGIN
Calculate(7, 9)
END StdExample.

## 12. Заголовок Открытого исходного кода

Модули, опубликованные под лицензией BlackBox Component Builder Open Source License, включают текстовый заголовок, оформленный по следующем образцу:

```
(**

project	= "BlackBox"
organization	= "www.oberon.ch"
contributors	= "Oberon microsystems"
version	= "System/Rsrc/About"
copyright	= "System/Rsrc/About"
license	= "Docu/BB-License"
purpose	= ""
changes	= "
- YYYYMMDD, nn, ...
"
issues	= "
- ...
"

**)
```
```